In [1]:
import pandas as pd

In [2]:
vanne_df = pd.read_excel("data/VANNE.xlsx")
cana = pd.read_excel("data/can.xlsx")



In [5]:
cana[cana["LONGUEUR_EN_M"] > 200].head(30)

,Unnamed: 0,ID_CANA,ID_NOEUD_1,ID_NOEUD_2,LONGUEUR_EN_M,DIAMETRE,COMMUNE,MATERIAU,LATITUDE_1,LONGITUDE_1,LATITUDE_2,LONGITUDE_2
36,36,2069546,122235,70598,424.230,100.0,Quesnoy-sur-Deûle,Fonte Ductile,50.711460,3.024731,50.710602,3.019008
131,131,1983684,36115,105086,238.107,60.0,Lys-lez-Lannoy,Fonte Ductile,50.668422,3.213432,50.669228,3.216545
165,165,1601959,18986,87453,333.023,100.0,Seclin,Fonte Ductile,50.547875,3.031380,50.545334,3.028928
174,174,596836,122266,122267,270.340,0.0,Lille,Inconnu,50.608874,3.032571,50.607920,3.034894
191,191,1605466,36127,70627,309.712,60.0,Comines,Fonte Ductile,50.750630,3.040026,50.749462,3.043775
192,192,1604438,18993,53473,217.260,60.0,Lys-lez-Lannoy,Fonte Ductile,50.673693,3.211568,50.675455,3.210310
194,194,116837,1902,1903,214.026,100.0,Ennetières-en-Weppes,Amiante Ciment,50.649977,2.915383,50.648918,2.912864
197,197,120002,105097,18994,329.387,250.0,La Chapelle-d'Armentières,Fonte Ductile,50.650048,2.915420,50.648432,2.911529
203,203,116805,18996,53478,351.420,80.0,Ennetières-en-Weppes,Amiante Ciment,50.647997,2.918792,50.645433,2.920829
311,311,2037572,53500,19019,206.237,200.0,Tourcoing,Fonte Ductile,50.709214,3.165094,50.710976,3.164247


In [1]:
import networkx as nx

from scripts.graph_coverage import compute_covered_pipelines


# Définissez MAX_DIAMETER_TO_CONSIDER pour le test
MAX_DIAMETER_TO_CONSIDER = 100

# Créez un exemple de graphe avec des attributs de longueur et de diamètre
G = nx.Graph()
G.add_edge("A", "B", length=50, diameter=90)  # Dans le rayon et le diamètre
G.add_edge("B", "C", length=100, diameter=80)  # Dans le rayon et le diamètre
G.add_edge("C", "D", length=60, diameter=110)  # Dans le rayon, mais diamètre trop grand
G.add_edge("A", "D", length=250, diameter=70)  # Longueur totale excédant max_distance

# Créez le dictionnaire edge_to_id pour identifier chaque arête
edge_to_id = {
    ("A", "B"): "pipe1",
    ("B", "C"): "pipe2",
    ("C", "D"): "pipe3",
    ("A", "D"): "pipe4",
}

# Spécifiez la distance maximale
max_distance = 200

# Appelez la fonction pour tester
covered_pipelines = compute_covered_pipelines(G, "A", edge_to_id, max_distance)

# Résultats attendus
expected_pipelines = {"pipe1", "pipe2"}  # "pipe1" et "pipe2" respectent les critères

# Assertion pour vérifier le résultat
assert (
    covered_pipelines == expected_pipelines
), f"Erreur : attendu {expected_pipelines}, obtenu {covered_pipelines}"

# Si le test passe, imprime une confirmation
print("Test réussi : les canalisations couvertes sont correctes.")

Test réussi : les canalisations couvertes sont correctes.


In [5]:
combined_nodes = pd.concat([df_commune["ID_NOEUD_1"], df_commune["ID_NOEUD_2"]])
vanne_in_cana = vanne[vanne["ID_NOEUD"].isin(combined_nodes)]
vanne_in_cana

,ID_NOEUD,ID_VANNE,DIAMETRE,ID_CANA_1,ID_CANA_2
1555,73766,152421,700.0,52303.0,52302.0
30510,119966,152420,400.0,52299.0,52301.0
30772,120150,152419,400.0,52297.0,52298.0
37256,33805,152422,400.0,52304.0,52300.0


In [24]:
combined_cana = pd.concat(
    [
        vanne_in_cana["ID_CANA_1"],
        vanne_in_cana["ID_CANA_2"],
    ],
)
combined_cana = combined_cana.drop(columns=["Unnamed: 0"], errors="ignore")
cana_in = cana[cana["ID_CANA"].isin(combined_cana)]
cana_in

0         139.0
1     2042362.0
2       69100.0
3       46069.0
4       23107.0
5      116100.0
6      597384.0
7      124091.0
8      116719.0
9      122849.0
10     122794.0
0         140.0
1     2042367.0
2       69098.0
3       46064.0
4       23099.0
5      116101.0
6      597327.0
7      123277.0
8      116722.0
9      122792.0
10     122851.0
dtype: float64

In [26]:
import os
import pandas as pd
from scripts.constants import GESTION_DATA_PATH
from scripts.preprocessing import create_columns

# Créer les colonnes dans le DataFrame vanne
vanne = create_columns(vanne)


# Vérifier si le fichier gestion_df existe
if os.path.exists(GESTION_DATA_PATH):
    # Lire le fichier Excel dans un DataFrame
    gestion_df = pd.read_excel(GESTION_DATA_PATH)

def combine_dataframe(vanne_df, gestion_df):
    # Effectuer une jointure gauche pour combiner les deux DataFrames
    vanne_df = pd.merge(
        vanne_df,
        gestion_df,
        on=["ID_NOEUD", "ID_VANNE"],
        how="left",
        suffixes=("", "_gestion"),
    )

    # Mettre à jour les colonnes de vanne_df avec les valeurs de gestion_df
    for col in ["BLOQUE", "FORCE", "LOCALISATION"]:
        vanne_df[col] = vanne_df[col + "_gestion"].combine_first(vanne_df[col])

    # Supprimer les colonnes temporaires ajoutées par la jointure
    vanne_df.drop(
        columns=[f"{col}_gestion" for col in ["BLOQUE", "FORCE", "LOCALISATION"]],
        inplace=True,
    )

    return vanne_df

# Afficher les 30 premières lignes du DataFrame résultant
vanne = combine_dataframe(vanne, gestion_df).head(30)  # Utilisez st.write si vous êtes dans Streamlit
vanne

,ID_NOEUD,ID_VANNE,DIAMETRE,ID_CANA_1,ID_CANA_2,BLOQUE,FORCE,LOCALISATION
0,50272,135808,200.0,139.0,140.0,1.0,1.0,
1,50273,2042365,80.0,2042362.0,2042367.0,1.0,1.0,
2,50279,157530,63.0,69100.0,69098.0,0.0,1.0,
3,50284,156831,60.0,46069.0,46064.0,1.0,1.0,
4,50290,145055,150.0,23107.0,23099.0,1.0,1.0,
5,50296,164081,200.0,116100.0,116101.0,1.0,1.0,
6,50297,150264,0.0,597384.0,597327.0,1.0,1.0,
7,50298,170878,80.0,124091.0,123277.0,1.0,1.0,
8,50302,169285,300.0,116719.0,116722.0,1.0,1.0,
9,50303,170619,200.0,122849.0,122792.0,1.0,1.0,


In [58]:
df_commune = cana[cana["COMMUNE"] == "Leforest"]
combined_nodes = pd.concat([df_commune["ID_NOEUD_1"], df_commune["ID_NOEUD_2"]])

vanne = create_columns(vanne_df)
vanne_in_cana = vanne[
        vanne["ID_NOEUD"].isin(combined_nodes)]

vanne_in_cana = vanne_in_cana[(vanne_in_cana["BLOQUE"] == 0) | (vanne_in_cana["FORCE"] == 1)]

vanne_nodes = set(vanne_in_cana["ID_NOEUD"])
vanne_nodes

{33805, 73766, 119966, 120150}